In [1]:
!pip install sqlalchemy pandas matplotlib 

StatementMeta(bf0c1f21-c5e2-40d5-ba38-e9717129a4b8, 0, 6, Finished, Available, Finished)

In [15]:
import sqlalchemy as sa
from urllib.parse import quote_plus

server   = "kafkasynapse14-ondemand.sql.azuresynapse.net"
database = "ghcn_processed"
username = "60313937@udst.edu.qa"           # your AAD UPN
password = quote_plus("Kashalaw147#")

engine = sa.create_engine(
    f"mssql+pyodbc://{username}:{password}@{server}:1433/{database}"
    "?driver=ODBC+Driver+18+for+SQL+Server"
    "&authentication=ActiveDirectoryPassword"
    "&Encrypt=yes&TrustServerCertificate=no"
)

with engine.connect() as conn:
    who = conn.execute(sa.text("SELECT SUSER_SNAME(), USER_NAME()")).fetchone()
print("Connected as:", who)











StatementMeta(bf0c1f21-c5e2-40d5-ba38-e9717129a4b8, 1, 20, Finished, Available, Finished)

Connected as: ('60313937@udst.edu.qa', 'dbo')


In [16]:
import pandas as pd
import sqlalchemy as sa

with engine.connect() as conn:
    station_id = conn.execute(sa.text("""
        SELECT TOP (1) Id
        FROM dbo.climate_parquet
        WHERE element IN ('TMAX','PRCP')
        GROUP BY Id
        HAVING COUNT(DISTINCT element) = 2
        ORDER BY Id;
    """)).scalar()

station_id


StatementMeta(bf0c1f21-c5e2-40d5-ba38-e9717129a4b8, 1, 21, Finished, Available, Finished)

'AE000041196'

In [17]:
q_monthly = sa.text("""
SELECT
  CONVERT(char(7), [date], 23) AS [date],                         -- YYYY-MM
  AVG(CASE WHEN element='TMAX' THEN CAST(data_value AS float) END)/10.0 AS avg_tmax_c,
  SUM(CASE WHEN element='PRCP' THEN CAST(data_value AS float) END)/10.0 AS precipitation_mm
FROM dbo.climate_parquet
WHERE Id = :sid
  AND element IN ('TMAX','PRCP')
GROUP BY CONVERT(char(7), [date], 23)
ORDER BY [date];
""")

with engine.connect() as conn:
    df_monthly = pd.read_sql(q_monthly, conn, params={"sid": station_id})

# Parse YYYY-MM and add year
df_monthly["month"] = pd.to_datetime(df_monthly["date"], format="%Y-%m")
df_monthly["year"]  = df_monthly["month"].dt.year

df_monthly.head()


StatementMeta(bf0c1f21-c5e2-40d5-ba38-e9717129a4b8, 1, 22, Finished, Available, Finished)

,date,avg_tmax_c,precipitation_mm,month,year
0,2023-01,24.140000,44.4,2023-01-01,2023
1,2023-02,28.273913,0.0,2023-02-01,2023
2,2023-03,30.296552,9.2,2023-03-01,2023
3,2023-04,31.738462,0.0,2023-04-01,2023
4,2023-05,39.184615,0.3,2023-05-01,2023


In [18]:
annual = (df_monthly
          .groupby("year", as_index=False)
          .agg(avg_tmax_c_annual=("avg_tmax_c", "mean"),
               precip_mm_annual=("precipitation_mm", "sum"))
          .sort_values("year")
          .reset_index(drop=True))
annual


StatementMeta(bf0c1f21-c5e2-40d5-ba38-e9717129a4b8, 1, 23, Finished, Available, Finished)

,year,avg_tmax_c_annual,precip_mm_annual
0,2023,34.977259,86.9
1,2024,33.996205,184.3
2,2025,36.167915,4.8


In [19]:
def year_diff(df_annual, newer, older):
    a = df_annual.set_index("year")
    if newer not in a.index or older not in a.index:
        return {"year_pair": f"{newer}-{older}", "delta_tmax_c": None, "delta_prcp_mm": None}
    return {
        "year_pair": f"{newer}-{older}",
        "delta_tmax_c": a.loc[newer, "avg_tmax_c_annual"] - a.loc[older, "avg_tmax_c_annual"],
        "delta_prcp_mm": a.loc[newer, "precip_mm_annual"]   - a.loc[older, "precip_mm_annual"],
    }

diffs = pd.DataFrame([
    year_diff(annual, 2024, 2023),
    year_diff(annual, 2025, 2024)  # 2025 may be incomplete or missing
])
diffs


StatementMeta(bf0c1f21-c5e2-40d5-ba38-e9717129a4b8, 1, 24, Finished, Available, Finished)

,year_pair,delta_tmax_c,delta_prcp_mm
0,2024-2023,-0.981054,97.4
1,2025-2024,2.171711,-179.5


In [20]:
your_station = station_id   # replace with e.g. "USW00094728"

with engine.connect() as conn:
    df_city = pd.read_sql(q_monthly, conn, params={"sid": your_station})

df_city["month"] = pd.to_datetime(df_city["date"], format="%Y-%m")
df_city["year"]  = df_city["month"].dt.year

annual_city = (df_city.groupby("year", as_index=False)
               .agg(avg_tmax_c_annual=("avg_tmax_c","mean"),
                    precip_mm_annual=("precipitation_mm","sum"))
               .sort_values("year"))
annual_city


StatementMeta(bf0c1f21-c5e2-40d5-ba38-e9717129a4b8, 1, 25, Finished, Available, Finished)

,year,avg_tmax_c_annual,precip_mm_annual
0,2023,34.977259,86.9
1,2024,33.996205,184.3
2,2025,36.167915,4.8
